In [1]:
import tensorflow as tf
tf.test.gpu_device_name()




'/device:GPU:0'

In [0]:
# !pip install keras

In [4]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.7MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

W0719 05:21:17.524216 140105013667712 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

title: Colab Notebooks, id: 14cWgjTJ2ahlliC6Gjo8CEyriIZGXL-1I
title: glove.840B.300d.txt, id: 10lbTTx5oR5eS_2OShO0kTA6j1esQCxfk
title: train.csv, id: 1wgk96WWngo5JwX4sGqSsXxo1wcx9pIgr
title: Scanned Documents.pdf, id: 1tjyPkgPxnxazxKEUb532sB3yvQkUeFW4
title: Scanned Documents.pdf, id: 1fZ-jE1JZ4bLxqXgARrTq84ovk6voU11R
title: Scanned Documents.pdf, id: 1X_MSMEPOcWzyCvHuh9Sr20DTWNQIY2m_
title: File_004.txt, id: 1lgIy_UgZGkH2obuGZ7Wyus8sViPbFvwB
title: Kaushik_Jagini_Certified_Data_Science_Specialist.pdf, id: 1dsaPScG4gbPLCmtVwaVJHUG_KpuiSHwO
title: Kaushik_Jagini_Certified_Data_Science_Specialist, id: 1Ft-zGSVx8uIFg8gXrmBYp6j9O7D9Y5EauO3ShIlEEDs
title: Kaushik_Jagini_Certified_Data_Science_Specialist.pdf, id: 0B-tuI-P1Bc25QjZWMktsZ1RQZmwyMTh4SjZxSzJnSTlxME1v
title: 13D6166F-EF66-4409-BF0E-D1F452DB979B.jpeg, id: 1ljnf3cy8xjIgDtNd_mMrzVLdcTIPI40x
title: Swing-Kaushik.mp4, id: 1RQt_07E-j_jcIlvF82AU6jPNPTcxaAHp
title: Untitled document, id: 1y_PkYvRIyphLUxqoedTFs4lEgJyhuiMCUCYVyif403s
title:

In [0]:

downloaded = drive.CreateFile({'id': '1wgk96WWngo5JwX4sGqSsXxo1wcx9pIgr'})
downloaded.GetContentFile('train.csv')

In [9]:
import os
os.listdir()

['.config', 'adc.json', 'train.csv', 'sample_data']

In [0]:
import pandas as pd
all_data = pd.read_csv("train.csv")

In [13]:
all_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_data['question_text'], all_data['target'], test_size=0.2, random_state=42)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk import word_tokenize

In [16]:
question_lens=[]
for question in all_data['question_text']:
    question_lens.append(len(word_tokenize(question)))

max(question_lens)

np.quantile(question_lens,0.90)

from keras.preprocessing.text import Tokenizer

max_len = 30
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train)
sequences_train = tok.texts_to_sequences(X_train)

X_train.head()

vocab_len=len(tok.index_word.keys())

vocab_len

from keras.preprocessing import sequence

sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

sequences_matrix_train

sequences_matrix_train[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     9,     3,  4780, 11233,     3,   130,
          27,   427, 11233], dtype=int32)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_len+1,300,input_length=max_len,mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
from keras.layers import Input, Embedding, LSTM, Activation, Dense, Dropout
from keras.models import Model

In [19]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])


W0719 05:46:06.143986 140105013667712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0719 05:46:06.160625 140105013667712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0719 05:46:06.162071 140105013667712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0719 05:46:06.466995 140105013667712 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instruct

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 300)           58857900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [0]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [25]:
model.fit(sequences_matrix_train,y_train,batch_size=500,epochs=10,validation_data=(sequences_matrix_test,y_test))

Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 362s 346us/step - loss: 0.0110 - acc: 0.9955 - val_loss: 0.3679 - val_acc: 0.9445
Epoch 2/10
1044897/1044897 [==============================] - 359s 344us/step - loss: 0.0093 - acc: 0.9963 - val_loss: 0.3916 - val_acc: 0.9442
Epoch 3/10
1044897/1044897 [==============================] - 359s 344us/step - loss: 0.0080 - acc: 0.9969 - val_loss: 0.3906 - val_acc: 0.9438
Epoch 4/10
1044897/1044897 [==============================] - 360s 344us/step - loss: 0.0071 - acc: 0.9972 - val_loss: 0.3988 - val_acc: 0.9445
Epoch 5/10
1044897/1044897 [==============================] - 357s 342us/step - loss: 0.0062 - acc: 0.9976 - val_loss: 0.4077 - val_acc: 0.9447
Epoch 6/10
1044897/1044897 [==============================] - 356s 341us/step - loss: 0.0054 - acc: 0.9980 - val_loss: 0.4235 - val_acc: 0.9452
Epoch 7/10
1044897/1044897 [==============================] - 357s 341us/step - los

In [0]:
predictions=model.predict(sequences_matrix_test)

In [0]:
from sklearn.metrics import roc_auc_score

In [28]:
roc_auc_score(y_test,predictions)

0.8945822544205431